In [19]:
import pandas as pd
import plotly.offline as py
from plotly import tools
import csv
import numpy as np

In [144]:
class patientModel():
    def __init__(self, dim1, dim2, dim3):
        self.dim = [dim1, dim2, dim3]
        ##quiz stores the prediction score
        self.quiz = np.array([['category A','category B','category C','category A and B','category A and C','category B and C'],[0,0,0,0,0,0]])##question
        ##reads in the xls file, adjest the path when running the programe
        data = pd.read_excel("/Users/s.li/Desktop/Cognitive Modelling/week2/training.xls", sheet_name='Sheet1')
        self.serialized_data = np.array([data['Dim 1'], data['Dim 2'], data['Dim 3'], data['Answer']])
        self.table = np.array([data['Dim 1'], data['Dim 2'], data['Dim 3'], data['Answer']])##stores weight value

        ##this is the weight that I assigned specific to this problem, which indicates how likely each label influencing the result
        ## npArray[[A,Y,X,Z,C],[X,Y,A,B],[C,Y,X,B]]
        ## each element calculated by(1-number of appearence/number of possible outcome in each )*the possibility of the most likely outcome

        self.dict = {'0':{'A': 0.696875, 'Y':  0.15, 'X': 0.205555556, 'Z': 0.65, 'C':0.95},
                '1':{'X': 0.225, 'Y': 0.225, 'A': 0.35, 'B': 0.516666667},
                '2':{'C': 0.58, 'Y': 0.322222222, 'X': 0.2875, 'B': 0.5375}}

##caluate each individual weight based on s parameter and each label  value
    def calculate_weights(self):
                
        ##this function trying to capture the focus when human trying to making certain predictions such as when they see A appears in category A they tend to think A is the answer
        def personalTempar(self):
            ##temp = random.uniform(1.5, 1.9)
            
            if self.dim[0]== 'A':
                self.quiz[1,0] = float(self.quiz[1,0]) + 0.3
                self.quiz[1,1] = float(self.quiz[1,0]) - 0.3
                self.quiz[1,2] = float(self.quiz[1,0]) - 0.3
            
            if self.dim[0]== 'C':
                self.quiz[1,2] = float(self.quiz[1,2]) + 0.3
                self.quiz[1,0] = float(self.quiz[1,2]) - 0.3
                self.quiz[1,1] = float(self.quiz[1,2]) - 0.3
                
            if self.dim[1]== 'B':
                self.quiz[1,1] = float(self.quiz[1,1]) + 0.2
                self.quiz[1,0] = float(self.quiz[1,1]) - 0.2
                self.quiz[1,2] = float(self.quiz[1,1]) - 0.2
                
            if self.dim[2]== 'B':
                self.quiz[1,1] = float(self.quiz[1,1]) + 0.2
                self.quiz[1,0] = float(self.quiz[1,1]) - 0.2
                self.quiz[1,2] = float(self.quiz[1,1]) - 0.2
            
            if self.dim[2]== 'c':
                self.quiz[1,2] = float(self.quiz[1,2]) + 0.2
                self.quiz[1,0] = float(self.quiz[1,2]) - 0.2
                self.quiz[1,1] = float(self.quiz[1,2]) - 0.2
                
        def calculateJunction(x,y,z):
            x = float(x) 
            y = float(y)
            if z == 3:
                junction = ((x + y) - x * y)/2 - 0.3
            else:
                junction = ((x + y) - x * y)/2 - 0.6
            
            return junction
           
        
        ##model making perdiction 
        def predictSingle(self,j):
            top = 0
            buttom = 0

            ##compute the first three single category prediction by mutiply the weight, the s parameter and the custom influence layer
            if j < 3:
                for i in range(0,16):
                    temp = self.table[0, i] * self.table[1, i] * self.table[2, i]
                    buttom = buttom + temp

                    if self.table[3, i] == self.quiz[0,j]:
                        top = top + temp

                self.quiz[1,j] = top/buttom
                
            ##compute the next 3 junction item by using

            if j == 3:
                self.quiz[1,j] = calculateJunction(self.quiz[1,0], self.quiz[1,1], j)

            if j == 4:
                self.quiz[1,j] = calculateJunction(self.quiz[1,0], self.quiz[1,2], j)

            if j == 5:
                self.quiz[1,j] = calculateJunction(self.quiz[1,1], self.quiz[1,2] ,j)
    
        

        for i in range(0,16):
            
            for x in range (0,3):
                if self.serialized_data[x, i] == self.dim[x]:
                    self.table[x,i] = 1
                else:
                    self.table[x,i] = (self.dict[str(x)][self.serialized_data[x,i]]) 
                    ##self.table[x,i] = self.dict[str(x)][self.serialized_data[x,i]]
                    
        for j in range(0,6):
            personalTempar(self)
            predictSingle(self,j)
            
  
        return self.quiz



In [145]:
setModels = [patientModel('A', 'B', 'Y'),patientModel('C', 'Y', 'B'),patientModel('Y', 'A', 'C'),patientModel('X', 'B', 'C'),patientModel('X', 'X', 'B')]
predictionOutput = [0]*30
counter = 0
##reformate the np arrays to single column array and format to 
for i in setModels:
    i.calculate_weights()
    for x in range(0,6):
        if x == 0:
            indexx = x + counter
        else:
            indexx = (counter + (x * 5))
            
        predictionOutput[indexx] = i.quiz[1,x]
    counter = counter+1
    print(i.quiz)

print(predictionOutput) 


[['category A' 'category B' 'category C' 'category A and B'
  'category A and C' 'category B and C']
 ['0.22052557055117' '0.42052557055117' '0.22052557055117'
  '-0.0258427501374' '-0.4037901930822' '-0.3258427501374']]
[['category A' 'category B' 'category C' 'category A and B'
  'category A and C' 'category B and C']
 ['0.44579587795519' '0.64579587795519' '0.44579587795519'
  '0.10184930775875' '-0.2535711044457' '-0.1981506922412']]
[['category A' 'category B' 'category C' 'category A and B'
  'category A and C' 'category B and C']
 ['0.22626841514243' '0.13551469832642' '0.57187618501303'
  '-0.1344397912749' '-0.2656264589425' '-0.2850533726763']]
[['category A' 'category B' 'category C' 'category A and B'
  'category A and C' 'category B and C']
 ['0.94231009074961' '1.14231009074961' '0.94231009074961'
  '0.14102896441022' '-0.1332020446647' '-0.0958950718896']]
[['category A' 'category B' 'category C' 'category A and B'
  'category A and C' 'category B and C']
 ['0.8735458040

In [147]:
##modify the output xls file's location when use
df1 = pd.DataFrame({'Data': predictionOutput})
writer = pd.ExcelWriter('/Users/s.li/Desktop/Cognitive Modelling/week2/output.xls', engine='xlsxwriter')
df1.to_excel(writer, sheet_name='Sheet1',
             startrow=1, startcol=1, header=False, index=False)